In [1]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

In [2]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

In [3]:
def step(item):
  prev_v, prev_p, next_v = item[0], item[1][0], item[1][1]
  new_p = prev_p.copy()
  new_p.append(next_v)
  return (next_v, new_p)

In [4]:
def reconstruct_path(x, y):
    paths = sc.parallelize([(x, [x])]).partitionBy(n)
    while True:
        reached = paths.filter(lambda i: i[0] == y)
        if reached.count() > 0:
            return reached.take(1)
        paths = paths.join(forward_edges, n).map(step).persist()

In [5]:
x = 12
y = 34
reconstructed_path = reconstruct_path(x, y)

In [6]:
print(','.join([str(i) for i in reconstructed_path[0][1]]))

12,422,53,52,107,20,23,274,34
